# Introduction

Notebook para fazer análise de modelos e de resultados de experimentos feitos e armazenados no Neptune

# Import Libraries

In [1]:
import os
import sys
import neptune.new as neptune
import zipfile
import shutil
import numpy as np
import pandas as pd

from datetime import datetime

from tqdm.notebook import tqdm

if '../..' not in sys.path:
    sys.path.insert(0, '../..')

import config

# Get Project Data

In [2]:
df = pd.read_csv('single_task_exps_data/icao-nets-training-2.csv')
print(df.shape)
df.head(23)

(23, 5)


,Id,Creation Time,Owner,Tags,props/icao_reqs
0,ICAO-111,2021-06-10 23:24:18,guilhermemg,"[""ground truths"",""binary_output"",""adamax"",""ink...",['ink_mark']
1,ICAO-110,2021-06-10 22:17:57,guilhermemg,"[""adamax"",""fvc"",""binary_output"",""washed_out"",""...",['washed_out']
2,ICAO-106,2021-06-09 23:57:07,guilhermemg,"[""adamax"",""fvc"",""not_aligned"",""vgg16"",""pixelat...",['pixelation']
3,ICAO-102,2021-06-09 21:48:11,guilhermemg,"[""fvc"",""red_eyes"",""not_aligned"",""vgg16"",""adama...",['red_eyes']
4,ICAO-96,2021-06-09 05:13:52,guilhermemg,"[""adamax"",""vgg16"",""blurred"",""binary_output"",""f...",['blurred']
5,ICAO-95,2021-06-09 04:09:55,guilhermemg,"[""binary_output"",""vgg16"",""skin_tone"",""adamax"",...",['skin_tone']
6,ICAO-92,2021-06-09 00:58:17,guilhermemg,"[""binary_output"",""fvc"",""hair_eyes"",""ground tru...",['hair_eyes']
7,ICAO-91,2021-06-08 23:54:22,guilhermemg,"[""vgg16"",""ground truths"",""adamax"",""fvc"",""binar...",['background']
8,ICAO-80,2021-06-05 18:12:20,guilhermemg,"[""fvc"",""vgg16"",""adamax"",""binary_output"",""groun...",['sh_head']
9,ICAO-74,2021-06-05 16:00:19,guilhermemg,"[""vgg16"",""binary_output"",""fvc"",""ground truths""...",['sh_face']


# Download de Modelos

In [3]:
def process_gt_names(gt_names):
    gt_names_dict = {'train_validation_test': gt_names.split(':')[-1]}
    train_val_test = gt_names_dict['train_validation_test']
    train_val_test = train_val_test[:-3]
    train_val_test = train_val_test[3:]
    return train_val_test


def download_model(exp_id):
    try:
        neptune_run = None
        print(f'Experiment ID: {exp_id}')

        neptune_run = neptune.init(project='guilhermemg/icao-nets-training-2', 
                                   api_token=config.NEPTUNE_API_TOKEN,
                                   run=exp_id)

        ds = neptune_run['props/gt_names'].fetch()
        alg = neptune_run['props/aligned'].fetch()
        req = neptune_run['props/icao_reqs'].fetch()

        ds = process_gt_names(ds)
        aligned = 'aligned' if bool(alg) == True else 'not_aligned'
        req = req[2:-2]
        
        print(f' Requisite: {req.upper()} | Dataset: {ds.upper()} | Aligned: {aligned}')
        
        dest_dir = f'prev_trained_models/single_task/{ds}_{aligned}/{req}/{exp_id}'
        print(f'Destination path: {dest_dir}')
        
        if os.path.exists(dest_dir):
            #shutil.rmtree(dest_dir)
            print('Model already downloaded!')
            print('-----------------------------')
            return
            
        os.makedirs(dest_dir)
        
        try:
            success_download = False
            neptune_run['artifacts/trained_model'].download(dest_dir)
            success_download = True

            with zipfile.ZipFile(os.path.join(dest_dir, 'trained_model.zip'), 'r') as zip_ref:
                zip_ref.extractall(dest_dir)

            os.remove(os.path.join(dest_dir, 'trained_model.zip'))
            shutil.move(os.path.join(dest_dir, 'trained_model', 'variables'), dest_dir)
            shutil.move(os.path.join(dest_dir, 'trained_model', 'saved_model.pb'), dest_dir)
            shutil.rmtree(os.path.join(dest_dir, 'trained_model'))

            print('.. Folders set')
        except Exception as ex:
            print(f'Experiment {exp_id} did not produce a trained model!')
        finally:
            if not success_download:
                shutil.rmtree(dest_dir)
            print('-----------------------------')
            return
            
    except Exception as e:
        raise e
    finally:
        if neptune_run is not None:
            neptune_run.stop()

            
for exp_id in df.Id.values:
    download_model(exp_id)            

Experiment ID: ICAO-111
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-111
 Requisite: INK_MARK | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/ink_mark/ICAO-111
.. Folders set
-----------------------------
Experiment ID: ICAO-110
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-110
 Requisite: WASHED_OUT | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/washed_out/ICAO-110
.. Folders set
-----------------------------
Experiment ID: ICAO-106
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-106
 Requisite: PIXELATION | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/pixelation/ICAO-106
.. Folders set
-----------------------------
Experiment ID: ICAO-102
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-102
 Requisite: RED_EYES | Dataset: FVC | Aligned: not_aligned
Destinati